## Построение эмбеддингов для рекомендаций по пользователям с реддита и оценка получившейся рекомендательной системы

In [2]:
import json
import random
import numpy as np
import pickle

In [31]:
data = []

for i in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
    data1 = json.load(open("Downloads/alexaprize-export-" + str(i) + ".json", "r"))
    data += data1

In [32]:
len(data)

8043

In [5]:
movie_list = [
    "What is the recent movie you've watched?",
    "What is your all-time favorite movie?",
    "What is the best movie you have ever seen?",
    "What is the scariest movie you have ever seen?",
    "What is the funniest movie you have ever seen?",
    "What is the most romantic movie you have ever seen?",
    "What movie you could watch over and over again?",
    "What is your favorite TV-series?"
]

In [6]:
sport_list = ["Do you like sports?",
                               "Do you like fitness?",
                               "Would you like to chat about sport?",
                                "Would you like to talk about your favorite athletes? ",
                                "Do you have a sports idol?",
                                "Well, Do you want to talk about sports competitions?"]

In [7]:
book_list = ["What is the last book you've read?",
"What is your favourite book?",
"What is book did impress you the most?",
"What is your favorite book?",
                         "Which book impressed you the most?",
                         "What is the longest book you have ever read?",
                         "What's a book you like to recommend to other people?",
                         "What is a book that was recommended to you?"]

In [8]:
games_list = ["do you love video games?"]

In [9]:
music_list = ["What music do you like?",
                                "Do you like listening to music?",
                                "What do you listen to?",
                                "Who is your favorite artist?",
                                "What is your favorite song?",
                                "What is your favorite album?",
                                "Do you have a favorite artist?"
                                ]

In [10]:
news_list = ["Would you like to hear the latest news?"]

In [11]:
animals_list = ["Do you like animals?",
                "Do you have pets?",
                "Would you like to talk about animals?",
                            "Let's chat about animals. Do you agree?",
                            "I'd like to talk about animals, would you?"
                            ]

In [12]:
food_list = ["Do you like cooking?",
                               "Do you think cooking is cool?",
                               "Many people say they adore cooking. Do you agree?",
                               "Do you think cooking is a great thing?",
                "Would you like to chat about food?",
                         "Would you like to talk about food?",
                         "I'd like to talk about food, would you?"]

In [13]:
travel_list = ["Do you like travel?",
                                     "What is your view on travel?",
                                     "What is your opinion of travel?",
                                     "Do you think travel is cool?",
                                     "Many people say they adore travel. Do you agree?",
                                     "Do you think travel is a great thing?",
                "Would you like to talk about travel?",
                               "Let's chat about travel adventures! Do you agree?",
                               "I'd like to talk about travel, would you?"
                               ]

In [14]:
science_list = ['Do you like science?', 'Do you think science is cool?', 'Many people say they adore science. Do you agree?', 'Do you think science is a great thing?', 'Would you like to chat about science?', 'Would you like to talk about science?', "I'd like to talk about science, would you?"]

In [15]:
gossip_list = ['Would you want to hear the latest gossip?', 'Are you interested in the latest gossip?', 'Would you be interested in the latest gossip?', 'What is your favourite celebrity?']

In [16]:
gamesdff_list = ["What video game are you playing in recent days?", "What is your favorite video game?"]

In [60]:
offer_sentence = {'game_cooperative_skill': games_list, 
                  'dff_movie_skill': movie_list, 
                  'book_skill': book_list, 
                  "news_api_skill": news_list, 
                  'dff_sport_skill': sport_list, 
                  'dff_music_skill': music_list, 
                  'dff_animals_skill': animals_list, 
                  'dff_travel_skill': travel_list,
                  'dff_food_skill': food_list,
                  'dff_science_skill': science_list,
                  'dff_gossip_skill': gossip_list}

In [58]:
scenario_skills = ['dff_movie_skill', 'book_skill', 'game_cooperative_skill', 'dff_travel_skill', 
 'dff_animals_skill', 'dff_food_skill', 'dff_sport_skill', 'dff_gossip_skill', 'dff_science_skill', 
 'dff_music_skill', "news_api_skill"]

topic2skill = {"Movies_TV": 'dff_movie_skill', "Music": 'dff_music_skill', "SciTech": 'dff_science_skill',
               "Literature": 'book_skill', "Travel_Geo": 'dff_travel_skill', "Celebrities": 'dff_gossip_skill',
               "Games": 'game_cooperative_skill', "Pets_Animals": 'dff_animals_skill', "Sports": 'dff_sport_skill',
               "Food_Drink": 'dff_food_skill', "News": "news_api_skill"}

def build_embedding(djson, l):
    current_emb = []
    for i, utt in enumerate(djson["utterances"]):
        if i % 2 == 1 and i < l:
            if utt.get("active_skill", "") in scenario_skills:
                current_emb.append(utt.get("active_skill", ""))
            elif i < l-1 and len(djson["utterances"][i+1].get("annotations", {}).get('cobot_topics', {}).get("text", [])) > 0:
                topic_list = djson["utterances"][i+1]["annotations"]['cobot_topics']["text"]
                added = 0
                for t in topic_list:
                    if len(topic2skill.get(t, "")) > 0:
                        added = 1
                        current_emb.append(topic2skill[t])
                if not added:
                    current_emb.append("")
            else:
                current_emb.append("")
    current_dict = {skill: 0 for skill in scenario_skills}
    emb = []
    for el in current_emb:
        if len(el) > 0:
            current_dict[el] += 1
    emb = [current_dict[skill]/len(current_emb) if len(current_emb) > 0 else 0 for skill in scenario_skills]
    return emb

In [49]:
scenario_skills = ['dff_movie_skill', 'book_skill', 'game_cooperative_skill', 'dff_travel_skill', 
 'dff_animals_skill', 'dff_food_skill', 'dff_sport_skill', 'dff_gossip_skill', 'dff_science_skill', 
 'dff_music_skill', "news_api_skill"]

topic2skill = {"Movies_TV": 'dff_movie_skill', "Music": 'dff_music_skill', "SciTech": 'dff_science_skill',
               "Literature": 'book_skill', "Travel_Geo": 'dff_travel_skill', "Celebrities": 'dff_gossip_skill',
               "Games": 'game_cooperative_skill', "Pets_Animals": 'dff_animals_skill', "Sports": 'dff_sport_skill',
               "Food_Drink": 'dff_food_skill', "News": "news_api_skill"}

In [23]:
def build_embedding(active_skills, cobot_topics):    

    skills_dict = {skill: 0 for skill in scenario_skills}
    
    for skill in active_skills:
        if skill in scenario_skills:
            skills_dict[skill] += 1
    
    for topic in cobot_topics:
        if topic in topic2skill.keys():
            skill = topic2skill[topic]
            skills_dict[skill] += 1
            
    total_skill = sum(skills_dict.values())

    embedding = [skills_dict[skill] / total_skill if total_skill > 0 else 0 for skill in scenario_skills]
    used_topics = [skill for skill in scenario_skills if skills_dict[skill] > 0]
    return embedding

In [26]:
embedding = build_embedding(active_skills, cobot_topics)

In [36]:
for e, s in zip(embedding, scenario_skills):
    if e > 0:
        print(s)

dff_movie_skill
dff_animals_skill
dff_music_skill
news_api_skill


In [33]:
TOP_V = 3
TOP_K = 3

scores = np.array(embedding).dot(np.array(database).T)
top_indices = np.argsort(scores)[::-1]
similarity_vector = np.sum(np.array([database[top_idx] for top_idx in top_indices[:TOP_V]]), 0)
candidate_topics_idx = np.argsort(similarity_vector)[::-1][:TOP_K]
print(candidate_topics_idx)
candidate_topics = [scenario_skills[idx] for idx in candidate_topics_idx] 

[11 10  9]


In [32]:
candidate_topics

['news_api_skill', 'dff_music_skill', 'dff_science_skill']

In [46]:
with open('reddit_embeddings.pickle', 'rb') as f:
    database = pickle.load(f)

In [ ]:
djson["utterances"][i+1].get("annotations", {}).get('cobot_topics', {}).get("text", []) > 0

In [8]:
dialogue = data[0]["utterances"]

In [14]:
data[0]["bot"]

{'id': '607faabd5584d8712cd34bae',
 'persona': {},
 'user_type': 'bot',
 'attributes': {}}

In [22]:
active_skills, cobot_topics = [], []
for utt in dialogue:
    active_skills.append(utt.get("active_skill", ""))
    cobot_topics += utt.get("annotations", {}).get('cobot_topics', {}).get("text", [])
active_skills = [skill for skill in active_skills if skill]

In [17]:
active_skills

['dff_friendship_skill',
 'dff_friendship_skill',
 'dff_friendship_skill',
 'news_api_skill',
 'last_chance_service',
 'news_api_skill',
 'news_api_skill',
 'news_api_skill',
 'intent_responder',
 'dff_music_skill',
 'knowledge_grounding_skill',
 'grounding_skill',
 'program_y_wide',
 'cobotqa',
 'convert_reddit',
 'program_y_wide',
 'program_y_dangerous',
 'program_y_dangerous',
 'dff_animals_skill',
 'dff_animals_skill',
 'program_y_wide',
 'dummy_skill',
 'alexa_handler']

In [16]:
topics

['Phatic',
 'Phatic',
 'Phatic',
 'Phatic',
 'News',
 'Phatic',
 'News',
 'Phatic',
 'Phatic',
 'Phatic',
 'Music',
 'Music',
 'Phatic',
 'Phatic',
 'Movies_TV',
 'Phatic',
 'Phatic',
 'Phatic',
 'Phatic',
 'Phatic',
 'Movies_TV',
 'Pets_Animals',
 'Phatic',
 'Sex_Profanity',
 'Phatic',
 'Phatic',
 'Movies_TV']

In [9]:
dialogue

[{'text': 'talk to me',
  'user': {'id': '1f56d30ae6814c5faa254fb1de79f589',
   'persona': {},
   'profile': {'name': None,
    'gender': None,
    'location': None,
    'birthdate': None,
    'occupation': None,
    'income_per_year': None,
    'home_coordinates': None,
    'work_coordinates': None},
   'user_type': 'human',
   'attributes': {'entities': {},
    'used_links': {},
    'disliked_skills': [],
    'dff_friendship_skill_state': {'history': {'0': 'State.SYS_HELLO'},
     'shared_memory': {},
     'dialogflow_state': '{"vars": {"__stack__": [], "__user_utterance__": "talk to me", "__system_state__": "State.SYS_HELLO", "__state__": "State.USR_HELLO_AND_CONTNIUE", "__converged__": "True", "__goal_return_state__": "None", "__selected_response__": "Hi, this is an Alexa Prize Socialbot! How are you?"}, "gates": {}, "state": "<__tuple__>\\"GREETING\\"<__tuple__>\\"State.USR_HELLO_AND_CONTNIUE\\""}',
     'previous_human_utter_index': 0}},
   'user_external_id': 'amzn1.ask.account.

In [62]:
yes_talk_pair = []
no_talk_pair = []

for dialogue in data:
    for i, utt in enumerate(dialogue['utterances']):
        current_state = "user" if i % 2 == 0 else "bot"
        for key in list(offer_sentence.keys()):
            for sent in offer_sentence[key]:
                if sent.lower() in utt["text"].lower() and i < len(dialogue['utterances']) - 1 and current_state == "bot" and key not in dialogue['utterances'][i-1]["text"]:
                    topic = key
                    answer = dialogue['utterances'][i+1]["text"]
                    if len(dialogue['utterances'][i+1]["annotations"].get("intent_catcher", "")) > 0:
                        yes_reaction = dialogue['utterances'][i+1]["annotations"]["intent_catcher"]["yes"]["detected"]
                        no_reaction = dialogue['utterances'][i+1]["annotations"]["intent_catcher"]["no"]["detected"]
                    else:
                        yes_reaction = 0
                        no_reaction = 0
                    if len(dialogue['utterances'][i+1]["annotations"].get('combined_classification', "")) > 0:
                        if len(dialogue['utterances'][i+1]["annotations"]['combined_classification'].get("sentiment_classification", "")) > 0:
                            yes_sentiment = bool(dialogue['utterances'][i+1]["annotations"]['combined_classification']["sentiment_classification"].get("positive", ""))
                            no_sentiment = bool(dialogue['utterances'][i+1]["annotations"]['combined_classification']["sentiment_classification"].get("negative", ""))
                    else:
                        yes_sentiment = 0
                        no_sentiment = 0
                    context_embedding = build_embedding(dialogue, i)
                    if yes_reaction or yes_sentiment:
                        yes_talk_pair.append((topic, 1, context_embedding))
                    if no_reaction or no_sentiment:
                        no_talk_pair.append((topic, 0, context_embedding))

In [63]:
len(yes_talk_pair)

1588

In [64]:
len(no_talk_pair)

1018

In [65]:
features = yes_talk_pair + no_talk_pair

In [66]:
random.shuffle(features)

In [67]:
len(features)

2606

In [26]:
topics = list(offer_sentence.keys())

In [80]:
count_topics1 = {topic: 0 for topic in topics}

for t in no_talk_pair:
    count_topics1[t[0]] += 1

In [69]:
count_topics

{'game_cooperative_skill': 129,
 'dff_gaming_skill': 0,
 'dff_movie_skill': 57,
 'book_skill': 51,
 'news_api_skill': 126,
 'dff_sport_skill': 135,
 'dff_music_skill': 8,
 'dff_animals_skill': 63,
 'dff_travel_skill': 83,
 'dff_food_skill': 80,
 'dff_science_skill': 141,
 'dff_gossip_skill': 145}

In [81]:
count_topics2 = {topic: 0 for topic in topics}

for t in yes_talk_pair:
    count_topics2[t[0]] += 1

In [83]:
for k in count_topics2.keys():
    print(k, count_topics1[k] + count_topics2[k])

game_cooperative_skill 350
dff_gaming_skill 0
dff_movie_skill 163
book_skill 162
news_api_skill 246
dff_sport_skill 241
dff_music_skill 24
dff_animals_skill 201
dff_travel_skill 260
dff_food_skill 256
dff_science_skill 336
dff_gossip_skill 367


In [71]:
count_topics

{'game_cooperative_skill': 221,
 'dff_gaming_skill': 0,
 'dff_movie_skill': 106,
 'book_skill': 111,
 'news_api_skill': 120,
 'dff_sport_skill': 106,
 'dff_music_skill': 16,
 'dff_animals_skill': 138,
 'dff_travel_skill': 177,
 'dff_food_skill': 176,
 'dff_science_skill': 195,
 'dff_gossip_skill': 222}

In [49]:
count_topics = {topic: 0 for topic in topics}

for t in yes_talk_pair:
    count_topics[t[0]] += 1

In [50]:
count_topics

{'games': 346,
 'movie': 334,
 'book': 171,
 'news': 212,
 'sport': 223,
 'music': 271,
 'animals': 609,
 'travel': 286,
 'food': 318}

In [30]:
import pickle

In [93]:
with open('dialog_embeddings.pickle', 'wb') as f:
    pickle.dump(features, f)

## Оценка

In [34]:
with open('reddit_embeddings.pickle', 'rb') as f:
    database = pickle.load(f)

In [35]:
scenario_skills = ['dff_movie_skill', 'book_skill', 'game_cooperative_skill', 'dff_gaming_skill', 'dff_travel_skill', 
 'dff_animals_skill', 'dff_food_skill', 'dff_sport_skill', 'dff_gossip_skill', 'dff_science_skill', 
 'dff_music_skill', "news_api_skill"]

In [48]:
len(features[0][-1])

12

In [87]:
f1_topics = {key: {"y_true":[], "y_pred": []} for key in scenario_skills}

for w, el in enumerate(features):
    print(el)
    scores = np.array(el[-1]).dot(np.array(database).T)
    top_indices = np.argsort(scores)[::-1][:5]
    similarity_vector = np.sum(np.array([database[top_id] for top_id in top_indices[:3]]), 0)
    candidate_topics_idx = np.argsort(similarity_vector)[::-1][:5]
    print(candidate_topics_idx)
    candidate_topics = [scenario_skills[idx] for idx in candidate_topics_idx] 
    f1_topics[el[0]]["y_true"].append(el[1])
    if el[0] in candidate_topics:
        f1_topics[el[0]]["y_pred"].append(1)
    else:
        f1_topics[el[0]]["y_pred"].append(0)
    if w % 20 == 0:
        print(w)

('dff_food_skill', 0, [0.1875, 0.03125, 0.015625, 0.015625, 0.109375, 0.03125, 0.015625, 0.046875, 0.25, 0.046875, 0.03125])
[ 8 10  9  7  6]
0
('dff_sport_skill', 0, [0.10810810810810811, 0.02702702702702703, 0.05405405405405406, 0.13513513513513514, 0.0, 0.02702702702702703, 0.0, 0.05405405405405406, 0.05405405405405406, 0.0, 0.02702702702702703])
[ 3 10  9  8  7]
('dff_animals_skill', 1, [0.21052631578947367, 0.0, 0.0, 0.2631578947368421, 0.0, 0.0, 0.0, 0.05263157894736842, 0.0, 0.0, 0.05263157894736842])
[ 3 10  9  8  7]
('dff_travel_skill', 1, [0.07352941176470588, 0.0, 0.0, 0.029411764705882353, 0.029411764705882353, 0.029411764705882353, 0.07352941176470588, 0.0, 0.0, 0.029411764705882353, 0.0])
[ 6 10  9  8  7]
('dff_food_skill', 0, [0.7777777777777778, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[0 8 4 2 6]
('game_cooperative_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[9 3 6 8 4]
('game_cooperative_skill', 0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

[ 8 10  9  7  6]
('dff_gossip_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[9 3 6 8 4]
('dff_gossip_skill', 1, [0.0, 0.0, 0.0, 0.2857142857142857, 0.0, 0.0, 0.2857142857142857, 0.0, 0.09523809523809523, 0.0, 0.0])
[ 3  6 10  9  8]
('game_cooperative_skill', 0, [0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[0 8 4 2 6]
180
('game_cooperative_skill', 0, [0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 2  5 10  9  8]
('news_api_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.07692307692307693, 0.0, 0.0, 0.0])
[5 7 0 8 3]
('dff_movie_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6, 0.0, 0.0, 0.0, 0.0])
[ 6 10  9  8  7]
('news_api_skill', 0, [0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07407407407407407, 0.14814814814814814, 0.0])
[ 1 10  9  8  7]
('dff_travel_skill', 1, [0.0, 0.0, 0.0, 0.08333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0])
[9 0 8 5 4]
('book_skill', 1, [0.2647058823529412, 0.0294117647

[ 3 10  9  8  7]
('dff_food_skill', 0, [0.037037037037037035, 0.037037037037037035, 0.07407407407407407, 0.07407407407407407, 0.05555555555555555, 0.05555555555555555, 0.018518518518518517, 0.0, 0.018518518518518517, 0.37037037037037035, 0.0])
[9 0 8 5 4]
('dff_gossip_skill', 0, [0.21666666666666667, 0.0, 0.0, 0.03333333333333333, 0.016666666666666666, 0.21666666666666667, 0.0, 0.016666666666666666, 0.016666666666666666, 0.016666666666666666, 0.0])
[5 0 9 8 4]
('game_cooperative_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.08333333333333333, 0.25, 0.0])
[ 9  5  8  3 10]
('dff_science_skill', 1, [0.0, 0.05, 0.0, 0.15, 0.0, 0.1, 0.1, 0.1, 0.0, 0.0, 0.0])
[ 3 10  9  8  7]
('dff_travel_skill', 1, [0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 3 10  9  8  7]
('dff_gossip_skill', 0, [0.2972972972972973, 0.32432432432432434, 0.02702702702702703, 0.05405405405405406, 0.05405405405405406, 0.0, 0.0, 0.0, 0.0, 0.02702702702702703, 0.0])
[ 1 10  9  8  7]
('game_cooperative_s

[ 9  5  8  3 10]
('dff_travel_skill', 0, [0.35714285714285715, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[0 8 4 3 2]
('game_cooperative_skill', 1, [0.13636363636363635, 0.0, 0.045454545454545456, 0.0, 0.0, 0.0, 0.18181818181818182, 0.045454545454545456, 0.0, 0.36363636363636365, 0.0])
[9 6 0 5 4]
('dff_gossip_skill', 1, [0.0, 0.5714285714285714, 0.0, 0.047619047619047616, 0.0, 0.047619047619047616, 0.0, 0.0, 0.047619047619047616, 0.0, 0.047619047619047616])
[ 1 10  9  8  7]
('dff_movie_skill', 0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.125, 0.0, 0.0625, 0.0, 0.0625])
[ 5  1  8 10  9]
('dff_movie_skill', 1, [0.06451612903225806, 0.0, 0.0, 0.06451612903225806, 0.0967741935483871, 0.0, 0.06451612903225806, 0.0, 0.0, 0.0, 0.0])
[ 4 10  9  8  7]
('dff_gossip_skill', 1, [0.0, 0.0, 0.0, 0.08333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 3 10  9  8  7]
('dff_science_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.45161290322580644, 0.0, 0.0, 0.03225806451612903, 0.0, 0.06451612903

[9 3 6 8 4]
('dff_science_skill', 1, [0.2631578947368421, 0.0, 0.0, 0.0, 0.0, 0.3684210526315789, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 5  1  8 10  9]
660
('dff_travel_skill', 0, [0.3076923076923077, 0.0, 0.0, 0.07692307692307693, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0, 0.07692307692307693, 0.0])
[0 3 4 8 2]
('book_skill', 1, [0.0, 0.0, 0.047619047619047616, 0.09523809523809523, 0.0, 0.0, 0.0, 0.047619047619047616, 0.0, 0.23809523809523808, 0.0])
[9 0 8 5 4]
('game_cooperative_skill', 1, [0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.5714285714285714, 0.0])
[ 9  1  4  0 10]
('news_api_skill', 0, [0.2692307692307692, 0.0, 0.038461538461538464, 0.34615384615384615, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 3 10  9  8  7]
('book_skill', 0, [0.04, 0.0, 0.04, 0.6, 0.0, 0.0, 0.0, 0.0, 0.04, 0.0, 0.08])
[ 3 10  9  8  7]
('dff_food_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0])
[ 6 10  9  8  7]
('dff_science_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

...

[ 5  1  8 10  9]
('dff_science_skill', 0, [0.02702702702702703, 0.0, 0.0, 0.0, 0.0, 0.05405405405405406, 0.02702702702702703, 0.08108108108108109, 0.10810810810810811, 0.32432432432432434, 0.0])
[9 8 4 1 0]
('dff_gossip_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.5, 0.0, 0.0])
[ 8 10  9  7  6]
('game_cooperative_skill', 1, [0.25, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 4 10  9  8  7]
('dff_gossip_skill', 1, [0.42105263157894735, 0.0, 0.0, 0.10526315789473684, 0.0, 0.05263157894736842, 0.10526315789473684, 0.0, 0.0, 0.05263157894736842, 0.10526315789473684])
[0 8 3 4 6]
('dff_science_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.48148148148148145, 0.0, 0.07407407407407407, 0.0, 0.037037037037037035, 0.0])
[ 5  1  8 10  9]
('dff_movie_skill', 0, [0.034482758620689655, 0.0, 0.0, 0.0, 0.6551724137931034, 0.0, 0.0, 0.0, 0.034482758620689655, 0.0, 0.0])
[ 4 10  9  8  7]
('news_api_skill', 0, [0.018867924528301886, 0.11320754716981132, 0.0, 0.0754716981132075

[ 6 10  9  8  7]
('dff_animals_skill', 1, [0.07142857142857142, 0.0, 0.0, 0.07142857142857142, 0.0, 0.30357142857142855, 0.0, 0.0, 0.017857142857142856, 0.21428571428571427, 0.0])
[ 5  1  8 10  9]
('dff_gossip_skill', 1, [0.0, 0.0, 0.0, 0.08333333333333333, 0.08333333333333333, 0.0, 0.0, 0.08333333333333333, 0.0, 0.08333333333333333, 0.0])
[ 3  4 10  9  8]
('dff_animals_skill', 1, [0.11538461538461539, 0.1346153846153846, 0.019230769230769232, 0.038461538461538464, 0.0, 0.3269230769230769, 0.0, 0.0, 0.07692307692307693, 0.0, 0.0])
[ 5  1  8 10  9]
('dff_travel_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[9 3 6 8 4]
1960
('news_api_skill', 1, [0.0, 0.09090909090909091, 0.0, 0.0, 0.09090909090909091, 0.09090909090909091, 0.0, 0.09090909090909091, 0.0, 0.0, 0.0])
[ 4  1  7 10  9]
('news_api_skill', 1, [0.23076923076923078, 0.10256410256410256, 0.0, 0.07692307692307693, 0.0, 0.1282051282051282, 0.02564102564102564, 0.05128205128205128, 0.02564102564102564, 0.0, 0.0]

[ 6 10  9  8  7]
('dff_gossip_skill', 1, [0.0, 0.07692307692307693, 0.0, 0.038461538461538464, 0.0, 0.0, 0.19230769230769232, 0.0, 0.4230769230769231, 0.0, 0.038461538461538464])
[ 8 10  9  7  6]
('game_cooperative_skill', 0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[9 3 6 8 4]
('game_cooperative_skill', 0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[9 3 6 8 4]
('dff_sport_skill', 0, [0.1, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0])
[ 4  9 10  8  7]
('dff_travel_skill', 0, [0.09090909090909091, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09090909090909091, 0.0])
[9 6 0 5 4]
('game_cooperative_skill', 0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06666666666666667])
[10  5  2  9  1]
('dff_gossip_skill', 1, [0.18181818181818182, 0.09090909090909091, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[0 8 2 4 3]
('dff_travel_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[9 3 6 8 4]
('dff_travel_skill', 0, [0.0, 0.0, 0.0, 0.0, 0

[10  5  2  9  1]
('dff_sport_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.9615384615384616, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 4 10  9  8  7]
('book_skill', 1, [0.2857142857142857, 0.03571428571428571, 0.0, 0.35714285714285715, 0.0, 0.0, 0.0, 0.10714285714285714, 0.03571428571428571, 0.0, 0.0])
[ 3 10  9  8  7]
('dff_food_skill', 1, [0.2857142857142857, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07142857142857142, 0.14285714285714285, 0.0, 0.0])
[0 8 4 6 7]
('dff_sport_skill', 0, [0.025, 0.15, 0.0, 0.1, 0.0, 0.025, 0.0, 0.075, 0.05, 0.025, 0.0])
[ 1 10  9  8  7]
('dff_gossip_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0])
[ 5  1  8 10  9]
('book_skill', 0, [0.09523809523809523, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0])
[ 5  1  8 10  9]
('dff_gossip_skill', 0, [0.07407407407407407, 0.2962962962962963, 0.0, 0.0, 0.0, 0.07407407407407407, 0.0, 0.0, 0.0, 0.07407407407407407, 0.0])
[ 1 10  9  8  7]
2300
('dff_sport_skill', 1, [0.1666666666

[ 5  1  8 10  9]
('dff_animals_skill', 1, [0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.11538461538461539, 0.0, 0.0, 0.038461538461538464, 0.0, 0.0])
[ 5  1  8 10  9]
('news_api_skill', 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0])
[ 8  9 10  7  6]
('dff_gossip_skill', 1, [0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.2857142857142857, 0.0, 0.0, 0.0, 0.07142857142857142, 0.0])
[ 5  1  8 10  9]
('dff_science_skill', 1, [0.0, 0.0, 0.03333333333333333, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.26666666666666666, 0.1])
[ 3 10  9  8  7]
('dff_travel_skill', 1, [0.0, 0.0, 0.022727272727272728, 0.022727272727272728, 0.4090909090909091, 0.0, 0.0, 0.0, 0.0, 0.022727272727272728, 0.0])
[ 4 10  9  8  7]
('dff_gossip_skill', 1, [0.03571428571428571, 0.6071428571428571, 0.03571428571428571, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10714285714285714, 0.0, 0.03571428571428571])
[ 1 10  9  8  7]
('dff_travel_skill', 0, [0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0])
[ 6 10  9  8  7]
('dff_foo

In [88]:
from sklearn.metrics import f1_score

In [89]:
for topic, preds in f1_topics.items():
    result = f1_score(f1_topics[topic]["y_true"], f1_topics[topic]["y_pred"], average="weighted")
    print(topic, result)

dff_movie_skill 0.4942493760020185
book_skill 0.3707200467763848
game_cooperative_skill 0.34519052711993886
dff_travel_skill 0.3985663817663818
dff_animals_skill 0.4525248632705391
dff_food_skill 0.3493283004258614
dff_sport_skill 0.49816883700383974
dff_gossip_skill 0.510649471811931
dff_science_skill 0.47803195352214967
dff_music_skill 0.4684684684684684
news_api_skill 0.47528900768459736
